## Assignment - PSOC  
Unit Commitmnet Table using Bruteforce solution  
`20K45A0215`  
`PRABHU KIRAN KONDA`

COST_FUNCTIONS:  
400, 6, 0.004  
500, 6.4, 0.002  

LIMITS:  
100 - 600  
85 - 800

In [1]:
import pandas as pd
import numpy as np
import itertools
from utils import *

In [2]:
data = pd.read_csv('load_data.csv')
data

,Time,Load
0,00 : 00 - 03 : 00,400
1,03 : 00 - 06 : 00,500
2,06 : 00 - 09 : 00,600
3,09 : 00 - 12 : 00,700
4,12 : 00 - 15 : 00,800
5,15 : 00 - 18 : 00,900
6,18 : 00 - 21 : 00,1000
7,21 : 00 - 24 : 00,1100


In [3]:
class UCTable:
    
    def __init__(self, cost_functions, limits, data, time):
        self.cost_functions  = cost_functions
        self.limits = limits
        self.data = data
        self.time = time
        self.units = len(cost_functions)
        
    
    def load_wise_table(self):
        combinations = [list(i) for i in itertools.product([0, 1], repeat=self.units)]
        combinations = combinations[1:]
        sort_combinations(combinations)
        
         # columns
        gens = [f'Gen{i+1}' for i in range(self.units)]
        pgens = [f'pgen{i+1}' for i in range(self.units)]
        cost_fns = [f'F{i+1}' for i in range(self.units)]
        cols = gens + pgens + cost_fns
        cols.append('total')
        
        df = pd.DataFrame(columns=list(cols), index=range(len(combinations)))
        for j in range(self.units):
            df[f'Gen{j+1}'] = [i[j] for i in combinations]
        
        for i in range(self.units):
            df.loc[df[f'Gen{i+1}'] == 0, (f'pgen{i+1}')] = 0
            df.loc[df[f'Gen{i+1}'] == 1, (f'pgen{i+1}')] = self.data['Load'][self.time] if int(self.data['Load'][self.time]) in self.limits[i] else 0
            if all(list(df.loc[i, pgens] == 0)):
                df.loc[i, (f'F{i+1}' for i in range(len(pgens)))] = 'unfeasible'
        return df     

In [4]:
cost_functions = [[400, 6, 0.004], [500, 6.4, 0.002]]
limits = [range(100, 601), range(75, 801)]
units = len(cost_functions)

In [5]:
from IPython.display import display

In [9]:
optimals = []
for i in range(len(data)):
    load = data['Load'][i]
    print(f'Load : {load}')
    df = UCTable(cost_functions, limits, data, i).load_wise_table()
    units = len(cost_functions)
    for j in range(units, len(df)):
        pgens_cost(j, load, df, cost_functions)

    for i in range(units):
        if df.iloc[i, 2*units:3*units].tolist() == ['unfeasible' for _ in range(units)]:
            continue
        costs = [0 for _ in range(units)]
        active_units_at_i = active_units(df)[i]
        for e in active_units_at_i:
            pgs = df.iloc[i, units:units*2].tolist()
            costs[e] = round(cost_functions[e][0] + cost_functions[e][1] * pgs[e] + cost_functions[e][2] * pgs[e]**2, 3)
            # print(pgs)
            # print(costs)
            costs.append(sum(costs))
            df.iloc[i, 2*units:(3*units)+1] = costs


    pgens = [f'pgen{i+1}' for i in range(units)]
    for i in range(units):
        if all(list(df.loc[i, pgens] == 0)):
            df.loc[i, (f'F{i+1}' for i in range(len(pgens)))] = 'unfeasible'
            df.iloc[i, -1] = 'unfeasible'
    vals = [e for e in df['total'].tolist() if not isinstance(e, str)]
    min_idx = df['total'].tolist().index(min(vals))
    optimals.append(df.iloc[min_idx, 0:units].tolist())
    display(df)

Load : 400


,Gen1,Gen2,pgen1,pgen2,F1,F2,total
0,0,1,0,400,0,3380.0,3380.0
1,1,0,400,0,3440.0,0,3440.0
2,1,1,166.625,233.25,1510.806,2101.611,3612.417


Load : 500


,Gen1,Gen2,pgen1,pgen2,F1,F2,total
0,0,1,0,500,0,4200.0,4200.0
1,1,0,500,0,4400.0,0,4400.0
2,1,1,200.0,300.0,1760.0,2600.0,4360.0


Load : 600


...

,Gen1,Gen2,pgen1,pgen2,F1,F2,total
0,0,1,0,0,unfeasible,unfeasible,unfeasible
1,1,0,0,0,unfeasible,unfeasible,unfeasible
2,1,1,333.375,566.75,2844.806,4769.611,7614.417


Load : 1000


,Gen1,Gen2,pgen1,pgen2,F1,F2,total
0,0,1,0,0,unfeasible,unfeasible,unfeasible
1,1,0,0,0,unfeasible,unfeasible,unfeasible
2,1,1,366.625,633.25,3137.406,5354.811,8492.217


Load : 1100


,Gen1,Gen2,pgen1,pgen2,F1,F2,total
0,0,1,0,0,unfeasible,unfeasible,unfeasible
1,1,0,0,0,unfeasible,unfeasible,unfeasible
2,1,1,400.0,700.0,3440.0,5960.0,9400.0


In [10]:
optimals = pd.DataFrame(optimals, columns=[f'G{i+1}' for i in range(units)])

In [13]:
uctable = pd.concat([data, optimals], axis=1)
uctable

,Time,Load,G1,G2
0,00 : 00 - 03 : 00,400,0,1
1,03 : 00 - 06 : 00,500,0,1
2,06 : 00 - 09 : 00,600,0,1
3,09 : 00 - 12 : 00,700,1,1
4,12 : 00 - 15 : 00,800,1,1
5,15 : 00 - 18 : 00,900,1,1
6,18 : 00 - 21 : 00,1000,1,1
7,21 : 00 - 24 : 00,1100,1,1
